In [ ]:
import numpy as np
import time
from tensorflow._api.v2.compat.v1 import ConfigProto
from tensorflow._api.v2.compat.v1 import InteractiveSession
from tensorflow import keras
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


def Normal(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    # Return the standard variable
    std = np.std(X_train, axis=(0, 1, 2, 3))

    # plus 0.000000000001 to avoid Integer
    X_train = (X_train - mean) / (std + 0.000000000001)
    X_test = (X_test - mean) / (std + 0.000000000001)

    return X_train, X_test


def LoadData():
    (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    (X_train, X_test) = Normal(X_train, X_test)

    #test_labels = tf.constant(Y_test)
    #train_labels = tf.constant(Y_train)

    classes = 10

    Y_train = to_categorical(Y_train, classes)
    Y_test = to_categorical(Y_test, classes)

    X_valid, X_train_new, Y_valid, Y_train_new = X_train[:5000], X_train[5000:], Y_train[:5000], Y_train[5000:]

    return X_train_new, X_valid, X_test, Y_train_new, Y_valid, Y_test


# Define Conv, Bn and Relu Layer
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()

        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                        kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)

        if (self.drop):
            layer = self.dropOut(layer)

        return layer


# Create the Model
class CNNModel(tf.keras.Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))

        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)

        self.dense1 = keras.layers.Dense(units=512, activation='relu',
                                         kernel_regularizer=tf.keras.regularizers.l2(0.0005))

        self.batchnorm = tf.keras.layers.BatchNormalization()

        self.dense2 = keras.layers.Dense(units=10)

        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs):
        model = self.conv1(inputs)
        model = self.conv2(model)
        model = self.maxPooling1(model)
        model = self.conv3(model)
        model = self.maxPooling2(model)

        model = self.flat(model)
        model = self.dropOut(model)

        model = self.dense1(model)
        model = self.batchnorm(model)
        model = self.dropOut(model)
        model = self.dense2(model)
        model = self.softmax(model)

        return model


# Create the Model
class CNNModel(tf.keras.Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))

        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)

        self.dense1 = keras.layers.Dense(units=512, activation='relu',
                                         kernel_regularizer=tf.keras.regularizers.l2(0.0005))

        self.batchnorm = tf.keras.layers.BatchNormalization()

        self.dense2 = keras.layers.Dense(units=10)

        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs):
        model = self.conv1(inputs)
        model = self.conv2(model)
        model = self.maxPooling1(model)
        model = self.conv3(model)
        model = self.maxPooling2(model)

        model = self.flat(model)
        model = self.dropOut(model)

        model = self.dense1(model)
        model = self.batchnorm(model)
        model = self.dropOut(model)
        model = self.dense2(model)
        model = self.softmax(model)

        return model


if __name__ == '__main__':
    print('tf.__version__:', tf.__version__)
    print('keras.__version__:', keras.__version__)

    training_epochs = 250
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)


    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))


    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    X_train_new, X_valid, X_test, Y_train_new, Y_valid, Y_test = LoadData()

    # Data Augment
    datagen = ImageDataGenerator(featurewise_center=True,samplewise_center=False, featurewise_std_normalization=True, samplewise_std_normalization=False, zca_whitening=False,rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,vertical_flip=False)

    print(datagen)
    datagen.fit(X_train_new)

    model = CNNModel()


    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=learning_rate,
        decay_steps=10000,
        decay_rate=0.96)
    optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

    #optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    #The real-time test test is only for the convenience of observation, so it does not affect the training process.
    # validation_data=(X_test, Y_test)

    t1 = time.time()
    model.fit(datagen.flow(X_train_new, Y_train_new, batch_size=batch_size), epochs=training_epochs, verbose=2,callbacks=[reduce_lr], steps_per_epoch=X_train_new.shape[0] // batch_size, validation_data=(X_valid, Y_valid))
    t2 = time.time()
    CNNfit = float(t2 - t1)
    print("Time taken: {} seconds".format(CNNfit))

/usr/local/lib/python3.9/dist-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


tf.__version__: 2.12.0
keras.__version__: 2.12.0
Epoch 1/250
351/351 - 34s - loss: 2.3939 - accuracy: 0.3844 - val_loss: 2.4982 - val_accuracy: 0.3662 - lr: 0.0999 - 34s/epoch - 96ms/step
Epoch 2/250
351/351 - 33s - loss: 2.0243 - accuracy: 0.4821 - val_loss: 1.9092 - val_accuracy: 0.5158 - lr: 0.0997 - 33s/epoch - 95ms/step
Epoch 3/250
351/351 - 31s - loss: 1.8727 - accuracy: 0.5283 - val_loss: 1.8762 - val_accuracy: 0.5428 - lr: 0.0996 - 31s/epoch - 87ms/step
Epoch 4/250
351/351 - 31s - loss: 1.7645 - accuracy: 0.5643 - val_loss: 1.8216 - val_accuracy: 0.5588 - lr: 0.0994 - 31s/epoch - 89ms/step
Epoch 5/250
351/351 - 31s - loss: 1.6737 - accuracy: 0.5922 - val_loss: 1.6215 - val_accuracy: 0.6010 - lr: 0.0993 - 31s/epoch - 89ms/step
Epoch 6/250
351/351 - 31s - loss: 1.6059 - accuracy: 0.6103 - val_loss: 1.7540 - val_accuracy: 0.5934 - lr: 0.0991 - 31s/epoch - 90ms/step
Epoch 7/250
351/351 - 31s - loss: 1.5410 - accuracy: 0.6270 - val_loss: 1.4942 - val_accuracy: 0.6554 - lr: 0.0990 - 

In [15]:
import pandas as pd
%pylab
%matplotlib inline

def tf_recall_score(y_true, y_pred):
    """
    micro: f1 score accross the classes, as 1
    macro: mean of f1 scores per class
    weighted: weighted average of f1 scores per class,
            weighted from the support of each class

    Args:
        y_true (Tensor): labels, with shape (batch, num_classes)
        y_pred (Tensor): model's predictions, same shape as y_true

    Returns:
        tuple(Tensor): (micro, macro, weighted)
                    tuple of the computed f1 scores
    """

    recalls = [0, 0, 0]

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    for i, axis in enumerate([None, 0]):
        TP = tf.math.count_nonzero(y_pred * y_true, axis=axis, dtype=tf.float32)
        FP = tf.math.count_nonzero(y_pred * (y_true - 1), axis=axis, dtype=tf.float32)
        FN = tf.math.count_nonzero((y_pred - 1) * y_true, axis=axis, dtype=tf.float32)

        #precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        recall = tf.cast(recall, tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)

        recalls[i] = tf.reduce_mean(recall)

    weights = tf.reduce_sum(y_true, axis=0)
    weights /= tf.reduce_sum(weights)
    weights = tf.cast(weights, tf.float64)

    recalls[2] = tf.reduce_sum(recall * weights)

    micro, macro, weighted = recalls
    return micro, macro, weighted

def tf_precision_score(y_true, y_pred):
    precisions = [0, 0, 0]

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    for i, axis in enumerate([None, 0]):
        TP = tf.math.count_nonzero(y_pred * y_true, axis=axis, dtype=tf.float32)
        FP = tf.math.count_nonzero(y_pred * (y_true - 1), axis=axis, dtype=tf.float32)
        FN = tf.math.count_nonzero((y_pred - 1) * y_true, axis=axis, dtype=tf.float32)

        precision = TP / (TP + FP)
        precision = tf.cast(precision, tf.float64)

        precisions[i] = tf.reduce_mean(precision)

    weights = tf.reduce_sum(y_true, axis=0)
    weights /= tf.reduce_sum(weights)
    weights = tf.cast(weights, tf.float64)

    precisions[2] = tf.reduce_sum(precision * weights)

    micro, macro, weighted = precisions
    return micro, macro, weighted

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [16]:
Y_proba = model.predict(X_test)
Y_pred = np.argmax(Y_proba, axis = 1)

classes = 10
labels = tf.cast(tf.constant(Y_test), dtype=tf.int32)

#Y_test_label = tf.one_hot(labels,classes)
print(labels) 

labels_ = tf.constant(Y_pred) 
Y_pred_label = tf.cast(tf.one_hot(labels_,classes), dtype=tf.int32)
print(Y_pred_label)

313/313 [==============================] - 1s 3ms/step
tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]], shape=(10000, 10), dtype=int32)
tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]], shape=(10000, 10), dtype=int32)


In [22]:
micro_recall, macro_recall, weighted_recall = tf_recall_score(labels, Y_pred_label)
#micro_recall, macro_recall, weighted_recall

micro_precision, macro_precision, weighted_precision = tf_precision_score(labels, Y_pred_label)
#micro_precision, macro_precision, weighted_precision
Y_test_notonehot = np.argmax(Y_test, axis=1)
print(Y_test, Y_pred)
metric = model.evaluate(X_test, Y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]] [3 8 8 ... 5 1 7]
313/313 [==============================] - 1s 4ms/step - loss: 1.0103 - accuracy: 0.8083


In [23]:
#DatFrame for accuracy, precision and recall on the test set
total_df = pd.DataFrame({
    'micro_precision': [micro_precision.numpy()],
    'macro_precision': [macro_precision.numpy()],
    'weighted_precision': [weighted_precision.numpy()],
    'micro_recall': [micro_recall.numpy()],
    'macro_recall': [macro_recall.numpy()],
    'weighted_recall': [weighted_recall.numpy()],
    'accuracy':[metric[1]]
})
total_df = total_df.style.set_caption('Test Set')

In [24]:
total_df

,micro_precision,macro_precision,weighted_precision,micro_recall,macro_recall,weighted_recall,accuracy
0,0.808300,0.831149,0.831149,0.808300,0.808300,0.808300,0.808300


In [25]:
predict_result = np.zeros((10, 10), dtype = int)

for i in range(10000):
    predict_result[ Y_test_notonehot[i] ][ Y_pred[i] ] += 1
#predict_result

pre = []
rcl = []
for i in range(10):
    pre.append( predict_result[i][i] / sum(predict_result.T[i]) )
    rcl.append( predict_result[i][i] / sum(predict_result[i]) )
    
classID_df = pd.DataFrame({
    'precision': pre,
    'recall': rcl
})
classID_df

,precision,recall
0,0.922535,0.786
1,0.954214,0.917
2,0.779519,0.746
3,0.820652,0.453
4,0.739564,0.815
5,0.856242,0.679
6,0.568366,0.981
7,0.944764,0.821
8,0.834934,0.956
9,0.890700,0.929
